In [1]:
from tqdm.auto import tqdm
from sentence_transformers import evaluation
import torch 
from transformers import BertModel
from transformers import BertTokenizer
from datasets import Dataset
import pandas as pd
from transformers.optimization import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt 
import numpy as np
import sklearn.model_selection
import sklearn
from sentence_transformers import SentenceTransformer, InputExample, losses, util

In [2]:
#df = pd.read_csv("/shared/3/projects/benlitterer/localNews/mergedNewsData/mergedNER.tsv", sep="\t")

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,6,9,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head(3)

,Unnamed: 0,article_id,sourcedomain_id,date,title,content,url_x,fips,source,description,...,predrt_3,national,id,author,url,published,published_utc,collection_utc,key,topics
0,0,andalusiastarnews--2020-04-04--Remember When: ...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Remember When: Shehan’s Polio story,Dan Shehan’s “My Polio Story” is continued in ...,https://www.andalusiastarnews.com/2020/04/03/r...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Remember_When:_Shehan’s_Poli...,"[('PERSON', 'Dan Shehan’s'), ('WORK_OF_ART', '..."
1,1,andalusiastarnews--2020-04-04--Veterans Founda...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Veterans Foundation salutes Vietnam veterans,"This past Sunday, March 29th marked National V...",https://www.andalusiastarnews.com/2020/04/03/v...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Veterans_Foundation_salutes_...,"[('DATE', 'This past Sunday, March 29th'), ('E..."
2,2,andalusiastarnews--2020-04-04--Governor issues...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Governor issues stay at home order,Governor Kay Ivey announced that a stay at hom...,https://www.andalusiastarnews.com/2020/04/03/g...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Governor_issues_stay_at_home...,"[('PERSON', 'Kay Ivey'), ('DATE', 'Saturday'),..."


In [39]:
modelPath = "/shared/3/projects/benlitterer/localNews/NetworkMVP/SBERTstockEval"
model = SentenceTransformer(modelPath, device="cuda:4")

testDf = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/enTestData.csv", sep="\t")

#compute embeddings for the test data and get correlation with ground truth 
text1 = list(testDf["text1"])
text2 = list(testDf["text2"])

embed1 = model.encode(text1)
embed2 = model.encode(text2)

cos_scores = [] 

for i in range(len(embed1)): 
    cos_scores.append(util.cos_sim(embed1[i], embed2[i]))
cos_scores[:2]

preds = [float(item) for item in cos_scores]
gt = (5 - testDf["Overall"]) / 4
np.corrcoef([preds, gt])


'\n#compute embeddings for the test data and get correlation with ground truth \ntext1 = list(testDf["text1"])\ntext2 = list(testDf["text2"])\n\nembed1 = model.encode(text1)\nembed2 = model.encode(text2)\n\ncos_scores = [] \n\nfor i in range(len(embed1)): \n    cos_scores.append(util.cos_sim(embed1[i], embed2[i]))\ncos_scores[:2]\n\npreds = [float(item) for item in cos_scores]\ngt = (5 - testDf["Overall"]) / 4\nnp.corrcoef([preds, gt])\n'

In [41]:
#try and encode just some of the articles 
testDf = df.head(1000)
textList = testDf["content"] 

embeddings = []
for article1 in tqdm(textList): 
    embeddings.append(model.encode(article1))

testDf["embedding"] = embeddings 
testDf

  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-41-4e9fa01e4a40>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testDf["embedding"] = embeddings


,Unnamed: 0,article_id,sourcedomain_id,date,title,content,url_x,fips,source,description,...,national,id,author,url,published,published_utc,collection_utc,key,topics,embedding
0,0,andalusiastarnews--2020-04-04--Remember When: ...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Remember When: Shehan’s Polio story,Dan Shehan’s “My Polio Story” is continued in ...,https://www.andalusiastarnews.com/2020/04/03/r...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Remember_When:_Shehan’s_Poli...,"[('PERSON', 'Dan Shehan’s'), ('WORK_OF_ART', '...","[-0.035961572, 0.07406131, 0.010887941, -0.021..."
1,1,andalusiastarnews--2020-04-04--Veterans Founda...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Veterans Foundation salutes Vietnam veterans,"This past Sunday, March 29th marked National V...",https://www.andalusiastarnews.com/2020/04/03/v...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Veterans_Foundation_salutes_...,"[('DATE', 'This past Sunday, March 29th'), ('E...","[-0.028507976, 0.06599003, 0.0063996348, -0.03..."
2,2,andalusiastarnews--2020-04-04--Governor issues...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Governor issues stay at home order,Governor Kay Ivey announced that a stay at hom...,https://www.andalusiastarnews.com/2020/04/03/g...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Governor_issues_stay_at_home...,"[('PERSON', 'Kay Ivey'), ('DATE', 'Saturday'),...","[-0.009479698, 0.049412683, 0.007799748, -0.04..."
3,3,andalusiastarnews--2020-04-03--Students prepa...,andalusiastarnews-andalusiastarnews.com,2020-04-03,Students prepare for online classes,Students across Covington County are scheduled...,https://www.andalusiastarnews.com/2020/04/03/s...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Students__prepare_for__onlin...,"[('GPE', 'Covington County'), ('DATE', 'Monday...","[-0.0054717152, 0.04881796, 0.006409988, -0.04..."
4,4,andalusiastarnews--2020-04-03--Local banks awa...,andalusiastarnews-andalusiastarnews.com,2020-04-03,"Local banks await final rules, guidance for pa...",As the novel coronavirus (COVID-19) disease co...,https://www.andalusiastarnews.com/2020/04/03/l...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Local_banks_await_final_rule...,"[('GPE', 'Alabama'), ('GPE', 'Covington County...","[-0.011190928, 0.018926142, -0.045841016, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,thestatejournalregister--2020-04-03--Trump bla...,thestatejournalregister-sj-r.com,2020-04-03,Trump blasts 3M as company says mask demand fa...,President Donald Trump is criticizing 3M over ...,https://www.sj-r.com/zz/news/20200403/trump-bl...,17167.0,thestatejournalregister,"Offers daily news, sports, features, and class...",...,False,NaN,NaN,NaN,NaN,NaN,NaN,thestatejournalregister_Trump_blasts_3M_as_com...,"[('PERSON', 'Donald Trump'), ('CARDINAL', '3')...","[-0.03251034, 0.028453153, -0.009553896, -0.01..."
996,996,thestatejournalregister--2020-04-03--Jared Kus...,thestatejournalregister-sj-r.com,2020-04-03,Jared Kushner makes coronavirus briefing appea...,WASHINGTON – White House senior advisor Jared ...,https://www.sj-r.com/zz/news/20200403/jared-ku...,17167.0,thestatejournalregister,"Offers daily news, sports, features, and class...",...,False,NaN,NaN,NaN,NaN,NaN,NaN,thestatejournalregister_Jared_Kushner_makes_co...,"[('GPE', 'WASHINGTON'), ('ORG', 'White House')...","[-0.025375726, 0.04648476, 0.0027397114, -0.02..."
997,997,thestatejournalregister--2020-04-03--Facebook ...,thestatejournalregister-sj-r.com,2020-04-03,"Facebook pledges $40M for small

In [51]:
#need to subset the data down a bit... 
#national = df[df["national"] == True]
sources = national["source"].unique()

#sanity check 
for item in toKeep: 
    if item not in sources: 
        print(item)
        
toKeep = ["vox", "cbsnews", "usatoday", "buzzfeednews", "businessinsider", "cbssports", "foxnews", "cnn", "cnbc", "huffingtonpost", "washingtonpost", "msnbc", "yahoonews", "thenewyorktimes", "fivethirtyeight", "forbes", "abcnews", "huffpost", "nytimes", "newyorkpost", "dailymail", "vanityfair"]

institutional = df[(df["national"] == False) | ((df["national"] == True) & (df["source"].isin(toKeep)))]

(1748548, 46)

In [57]:
institutional["date"] = pd.to_datetime(institutional["date"])


<ipython-input-57-0257872deb5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  institutional["date"] = pd.to_datetime(institutional["date"])


In [58]:
institutional[["source", "date"]]

,source,date
0,andalusiastarnews,2020-04-04
1,andalusiastarnews,2020-04-04
2,andalusiastarnews,2020-04-04
3,andalusiastarnews,2020-04-03
4,andalusiastarnews,2020-04-03
...,...,...
4626098,thenewyorktimes,2021-08-09
4626099,thenewyorktimes,2021-09-24
4626100,thenewyorktimes,2021-10-15
4626101,thenewyorktimes,2021-07-13


In [61]:
institutionalSubset = institutional[(institutional["date"] >= "2021-05-01") & (institutional["date"] <= "2021-09-01")]
institutionalSubset.head(3)

,Unnamed: 0,article_id,sourcedomain_id,date,title,content,url_x,fips,source,description,...,predrt_3,national,id,author,url,published,published_utc,collection_utc,key,topics
757482,757482,andalusiastarnews--2021-05-10--Ivey announces ...,andalusiastarnews-andalusiastarnews.com,2021-05-10,Ivey announces end of participation in all Fed...,Governor Kay Ivey announced today that Alabama...,https://www.andalusiastarnews.com/2021/05/10/i...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Ivey_announces_end_of_partic...,"[('PERSON', 'Kay Ivey'), ('DATE', 'today'), ('..."
757483,757483,andalusiastarnews--2021-05-08--Arrest report f...,andalusiastarnews-andalusiastarnews.com,2021-05-08,"Arrest report for Sat., May 8","\n• Houston Kervin, 34, for failure to appear,...",https://www.andalusiastarnews.com/2021/05/08/a...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,"andalusiastarnews_Arrest_report_for_Sat.,_May_8","[('DATE', '34'), ('PERSON', 'Melinda Manuel'),..."
757484,757484,andalusiastarnews--2021-05-07--Obituary for Ca...,andalusiastarnews-andalusiastarnews.com,2021-05-07,Obituary for Cathy Edmondson,"Cathy Reeves Edmondson, age 59, of Florala pas...",https://www.andalusiastarnews.com/2021/05/07/o...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Obituary_for_Cathy_Edmondson,"[('PERSON', 'Cathy Reeves Edmondson'), ('DATE'..."


In [64]:
print(max(institutionalSubset["date"]))
print(min(institutionalSubset["date"]))
print(institutionalSubset[institutionalSubset["national"] == False].shape)
print(institutionalSubset[institutionalSubset["national"] == True].shape)

2021-09-01 00:00:00
2021-05-01 00:00:00
(249659, 46)
(59286, 46)


In [ ]:
#WHAT WAS ACTUALLY RUN TO PRODUCE OUTPUT IN "getEmbeddings.py" SCRIPT 
#for copying into vim script 
from tqdm.auto import tqdm
from sentence_transformers import evaluation
import torch 
from transformers import BertModel
from transformers import BertTokenizer
from datasets import Dataset
import pandas as pd
from transformers.optimization import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt 
import numpy as np
import sklearn.model_selection
import sklearn
from sentence_transformers import SentenceTransformer, InputExample, losses, util

#we can't have any na rows when calculating embeddings on content 
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/mergedNewsData/mergedNER.tsv", sep="\t").dropna(subset=["content"])

#get only institutional national data 
toKeep = ["vox", "cbsnews", "usatoday", "buzzfeednews", "businessinsider", "cbssports", "foxnews", "cnn", "cnbc", "huffingtonpost", "washingtonpost", "msnbc", "yahoonews", "thenewyorktimes", "fivethirtyeight", "forbes", "abcnews", "huffpost", "nytimes", "newyorkpost", "dailymail", "vanityfair"]
institutional = df[(df["national"] == False) | ((df["national"] == True) & (df["source"].isin(toKeep)))]

#get may-september 2021 data 
institutionalSubset = institutional[(institutional["date"] >= "2021-05-01") & (institutional["date"] <= "2021-09-01")]

#sanity check 
print(max(institutionalSubset["date"]))
print(min(institutionalSubset["date"]))
print(institutionalSubset[institutionalSubset["national"] == False].shape)
print(institutionalSubset[institutionalSubset["national"] == True].shape)

modelPath = "/shared/3/projects/benlitterer/localNews/NetworkMVP/SBERTstockEval"
model = SentenceTransformer(modelPath, device="cuda:4")

outPath = "/shared/3/projects/benlitterer/localNews/NetworkMVP/dataWithEmbeddings.tsv"

df = institutionalSubset

#get content and put it in a list for encoding 
textList = df["content"] 

embeddings = []
for article in tqdm(textList): 
    embeddings.append(model.encode(article))
    
df["embedding"] = embeddings 
df.to_csv(outPath, sep="\t", quoting=csv.QUOTE_NONNUMERIC)